### 1) import datasets

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/" , one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### 2) initializing weights and biases

In [2]:
input_width = 28 
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024 # of units
n_out = 10      # ofunits

input_size_to_hidden = (input_width // (max_pool1_k * max_pool2_k)) * (input_height // (max_pool1_k * max_pool2_k)) * n_conv2

In [3]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k , conv1_k , input_channels , n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k , conv2_k , n_conv1 , n_conv2])) ,
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden , n_hidden])) ,
    "wo"  : tf.Variable(tf.random_normal([n_hidden , n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])) , 
    "bc2" : tf.Variable(tf.random_normal([n_conv2])) , 
    "bh1" : tf.Variable(tf.random_normal([n_hidden])) , 
    "bo" : tf.Variable(tf.random_normal([n_out]))
}

### 3) Forward Propagation

In [4]:
def conv(x , weights , bias , strides = 1):
    out = tf.nn.conv2d(x , weights , padding = "SAME" , strides = [1 , strides , strides , 1])
    out = tf.nn.bias_add(out , bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x , k = 2):
    return tf.nn.max_pool(x , padding = "SAME" , ksize = [1 , k , k , 1] , strides = [1 , k , k , 1])

In [5]:
# forward
def cnn(x , weights , biases , keep_prob):
    ## -1 is for infer the size , it will automatically fill the first size
    x = tf.reshape(x , shape = [ -1 , input_height , input_width , input_channels])
        
    ## cnn part .....
    conv1 = conv(x , weights["wc1"] , biases["bc1"] , stride_conv1)
    conv1_pool = maxpooling(conv1 , max_pool1_k)
    
    conv2 = conv(conv1_pool , weights["wc2"] , biases["bc2"] , stride_conv2)
    conv2_pool = maxpooling(conv2 , max_pool2_k)
    
    ## reshape to original format
    hidden_input = tf.reshape(conv2_pool ,shape = [-1 , input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input , weights["wh1"]) , biases["bh1"])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout , keep_prob)
    output = tf.add(tf.matmul(hidden_output , weights["wo"]) , biases["bo"])
    
    return output
    

In [6]:
x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y))

In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

901789.399493888
40219.36001363397
23956.219490707444
16349.599766741274
11257.720092945483
10140.574692701013
9217.12903651474
9513.836469548485
6310.271248396497
5461.271398638338
5901.799012111947
5251.687190786591
4439.914180814047
4540.2383428836965
4563.3356049115255
3689.811675340212
3045.699203925539
2913.088819910705
2797.5322378471833
2424.6886386664305
2948.518144794079
2493.4562018529277
1769.5681861084283
2487.770146941719
2037.0781651122234


In [11]:
predictions = tf.argmax(pred , 1)
correct_labels = tf.argmax(y , 1)
correct_predictions = tf.equal(predictions , correct_labels)
predictions , correct_preds = sess.run([predictions , correct_predictions] , feed_dict = {
    x : mnist.test.images ,
    y : mnist.test.labels ,
    keep_prob : 1.0
})
correct_preds.sum()

9850

### 4) Adding dropout Layer